## SVM+DECT

In [4]:
import torch
from torch import nn
from torch_geometric.data import Data, Batch
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from wect import ECTLayer, ECTConfig  

# Set device
DEVICE = "cpu"

# Function to load the Digits dataset
def load_digits_data():
    digits = load_digits()
    X, y = digits.data, digits.target
    return X, y

# Define RBF kernel function for DECT loss
def rbf_kernel(x1, x2, gamma=1.0):
    distance = torch.cdist(x1, x2, p=2)
    return torch.exp(-gamma * distance ** 2)

# Custom DECT loss class (adjusted for binary illustration in a multi-class context)
class DECTLoss(nn.Module):
    def __init__(self, gamma=1.0):
        super(DECTLoss, self).__init__()
        self.gamma = gamma

    def forward(self, ect_features, labels):
        kernel_matrix = rbf_kernel(ect_features, ect_features, self.gamma)
        # Labels should be -1 and 1 for binary hinge loss
        margins = torch.matmul(kernel_matrix, labels.float())
        loss = torch.mean(torch.clamp(1 - margins, min=0))
        return loss

# ECT feature extractor using original features
class ECTFeatureExtractor(nn.Module):
    def __init__(self, ect_config, directions):
        super(ECTFeatureExtractor, self).__init__()
        self.ect_layer = ECTLayer(ect_config, v=directions)

    def forward(self, batch):
        ect_features = self.ect_layer(batch)  # Uses batch.x (original features)
        return ect_features

# Load and preprocess the Digits dataset
X, y = load_digits_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Convert to PyTorch Geometric format
train_data = [Data(x=X_train[i:i+1]) for i in range(len(X_train))]
train_batch = Batch.from_data_list(train_data)

test_data = [Data(x=X_test[i:i+1]) for i in range(len(X_test))]
test_batch = Batch.from_data_list(test_data)

# Set ECT parameters
num_features = 64  # Digits dataset has 64 features (8x8 images)
num_thetas = 128
v = torch.randn(num_features, num_thetas)  # Random directions in 64D

ect_config = ECTConfig(
    ect_type="points",
    bump_steps=128,
    radius=1.0,
    normalized=False,
    fixed=True,
)

# Extract ECT features
ect_extractor = ECTFeatureExtractor(ect_config=ect_config, directions=v).to(DEVICE)
ect_features_train = ect_extractor(train_batch.to(DEVICE)).detach().cpu().numpy().reshape(len(y_train), -1)
ect_features_test = ect_extractor(test_batch.to(DEVICE)).detach().cpu().numpy().reshape(len(y_test), -1)

# Train SVM with RBF kernel
svm_model = SVC(kernel="rbf", C=1.0, gamma="scale")
svm_model.fit(ect_features_train, y_train.numpy())

# Evaluate SVM predictions
predicted_labels = svm_model.predict(ect_features_test)
accuracy = accuracy_score(y_test.numpy(), predicted_labels)
print(f"Test Accuracy: {accuracy:.2%}")

# Compute DECT loss for illustration (class 0 vs. rest)
y_train_binary = (y_train == 0).long() * 2 - 1  # 1 if y==0, else -1
dect_loss_fn = DECTLoss(gamma=1.0)
dect_loss = dect_loss_fn(torch.tensor(ect_features_train), y_train_binary)
print(f"DECT Loss (for class 0 vs. rest): {dect_loss.item():.4f}")

# Visualize the first two ECT features with predicted labels
plt.figure(figsize=(8, 6))
plt.scatter(ect_features_test[:, 0], ect_features_test[:, 1], c=predicted_labels, cmap="viridis", alpha=0.7)
plt.title("SVM Decision Boundaries (Using ECT Features) - Digits Dataset")
plt.xlabel("ECT Feature 1")
plt.ylabel("ECT Feature 2")
plt.colorbar(label="Predicted Digit")
plt.show()

AttributeError: 'GlobalStorage' object has no attribute 'node_weights'

## NN+DECT

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Batch
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

# Generate random directions for ECT projection
def generate_random_directions(num_directions, dim):
    directions = torch.randn(dim, num_directions)
    directions = directions / torch.norm(directions, dim=0, keepdim=True)
    return directions

# ECT Layer
class ECTLayer(nn.Module):
    def __init__(self, v):
        super(ECTLayer, self).__init__()
        self.v = v  # Random directions

    def forward(self, batch):
        nh = batch.x @ self.v  # Project features onto directions
        return nh

# ECT Classifier
class ECTClassifier(nn.Module):
    def __init__(self, dim, num_directions, hidden_dim, num_classes):
        super(ECTClassifier, self).__init__()
        self.v = generate_random_directions(num_directions, dim)
        self.ect_layer = ECTLayer(self.v)
        self.fc1 = nn.Linear(num_directions, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, batch):
        ect_features = self.ect_layer(batch)
        ect_features = ect_features.view(-1, ect_features.size(1))
        x = F.relu(self.fc1(ect_features))
        x = self.fc2(x)
        return x

# Load and prepare data
def load_digits_data():
    data = load_digits()
    X = data.data
    y = data.target
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    train_data = [Data(x=torch.tensor(X_train[i:i+1], dtype=torch.float32),
                       y=torch.tensor([y_train[i]], dtype=torch.long))
                  for i in range(len(X_train))]
    test_data = [Data(x=torch.tensor(X_test[i:i+1], dtype=torch.float32),
                      y=torch.tensor([y_test[i]], dtype=torch.long))
                 for i in range(len(X_test))]
    return train_data, test_data

# Training function
def train(model, train_data, num_epochs=200):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        model.train()
        batch = Batch.from_data_list(train_data)
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()

# Evaluation function
def evaluate(model, data_list):
    model.eval()
    batch = Batch.from_data_list(data_list)
    with torch.no_grad():
        out = model(batch)
        pred = out.argmax(dim=1)
        correct = (pred == batch.y).sum().item()
        accuracy = correct / len(batch.y)
    return accuracy

# Main execution
def main():
    # Parameters
    dim = 64  # Feature dimension for Digits dataset
    num_directions = 128
    hidden_dim = 64
    num_classes = 10  # Digits 0-9
    num_epochs = 200

    # Load data
    train_data, test_data = load_digits_data()

    # Initialize and train model
    model = ECTClassifier(dim, num_directions, hidden_dim, num_classes)
    train(model, train_data, num_epochs)

    # Compute accuracies
    train_accuracy = evaluate(model, train_data)
    test_accuracy = evaluate(model, test_data)

    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")

if __name__ == "__main__":
    main()

Training Accuracy: 1.0000
Test Accuracy: 0.9833
